In [101]:
# import cudaq
import numpy as np
import sys
import os

notebook_dir = '/workspace/tutorial_notebook'
if os.path.exists(notebook_dir) and notebook_dir not in sys.path:
    sys.path.insert(0, notebook_dir)


In [102]:
labs_count = 0
def labs_energy(s):
    """
    Compute the LABS energy function E(s) = sum_{k=1}^{N-1} C_k^2
    where C_k = sum_{i=1}^{N-k} s_i * s_{i+k}

    Args:
        s: Binary sequence with values in {-1, +1}

    Returns:
        Energy value (lower is better)
    """
    N = len(s)
    energy = 0
    for k in range(1, N):
        C_k = sum(s[i] * s[i + k] for i in range(N - k))
        energy += C_k ** 2
    global labs_count
    labs_count += 1
    if labs_count == 100000:
        print("Reached 100,000 energy evaluations, exiting.")
    return energy

In [103]:
def merit_factor(s, E) -> float:
    N = len(s)
    if E == 0:
        return float('inf')
    return N * N / (2 * E)

In [104]:

def combine(p1, p2):
    """
    Single-point crossover of two parent sequences (Algorithm 3 from paper).

    Args:
        p1: First parent sequence
        p2: Second parent sequence

    Returns:
        Child sequence combining p1[0:k] and p2[k:N]
    """
    N = len(p1)
    k = np.random.randint(1, N)  # cut point in {1, ..., N-1}
    return np.concatenate([p1[:k], p2[k:]])


In [105]:
def mutate(s, p_mut):
    """
    Probabilistic bit-flipping mutation (Algorithm 3 from paper).

    Args:
        s: Input sequence
        p_mut: Probability of flipping each bit

    Returns:
        Mutated sequence
    """
    s = s.copy()
    for i in range(len(s)):
        if np.random.random() < p_mut:
            s[i] = -s[i]  # flip +1 <-> -1
    return s


In [106]:
def tabu_search(s, max_iter=100, tabu_tenure=7):
    """
    Tabu search: a modified greedy local search that maintains a tabu list
    to avoid cycling back to recently visited solutions.

    Args:
        s: Starting sequence
        max_iter: Maximum number of iterations
        tabu_tenure: Number of iterations a move stays tabu

    Returns:
        best_s: Best sequence found
        best_energy: Energy of best sequence
    """
    s = s.copy()
    best_s = s.copy()
    best_energy = labs_energy(s)
    current_energy = best_energy
    tabu_list = {}  # position -> iteration when it becomes non-tabu

    for iteration in range(max_iter):
        # Find best non-tabu neighbor (single bit flip)
        best_neighbor = None
        best_neighbor_energy = float('inf')
        best_flip_pos = -1

        for i in range(len(s)):
            # Check if move is tabu (unless it leads to aspiration criterion)
            is_tabu = tabu_list.get(i, 0) > iteration

            # Compute neighbor energy
            neighbor = s.copy()
            neighbor[i] = -neighbor[i]
            neighbor_energy = labs_energy(neighbor)

            # Accept if not tabu, or if it beats the best (aspiration criterion)
            if (not is_tabu) or (neighbor_energy < best_energy):
                if neighbor_energy < best_neighbor_energy:
                    best_neighbor = neighbor
                    best_neighbor_energy = neighbor_energy
                    best_flip_pos = i

        if best_neighbor is None:
            break

        # Move to best neighbor
        s = best_neighbor
        current_energy = best_neighbor_energy
        tabu_list[best_flip_pos] = iteration + tabu_tenure

        # Update global best if improved
        if best_neighbor_energy < best_energy:
            best_s = s.copy()
            best_energy = best_neighbor_energy

    return best_s, best_energy

In [115]:
def memetic_tabu_search(N, pop_size=10, max_generations=100,
                        p_mut=0.1, p_combine=0.5, target_energy=None,
                        tabu_max_iter=100, tabu_tenure=7, verbose=False,
                        initial_population=None):
    """
    Memetic Tabu Search (MTS) for the LABS problem.

    Args:
        N: Sequence length
        pop_size: Population size
        max_generations: Maximum number of generations
        p_mut: Mutation probability per bit
        p_combine: Probability of combining vs sampling
        target_energy: Stop if this energy is reached (optional)
        tabu_max_iter: Max iterations for tabu search
        tabu_tenure: Tabu tenure for tabu search
        verbose: Print progress if True
        initial_population: Optional list of initial sequences

    Returns:
        best_s: Best sequence found
        best_energy: Energy of best sequence
        population: Final population
        energies: Energies of final population
    """
    # Initialize population with values in {-1, +1}
    if initial_population is not None:
        population = [np.array(s) for s in initial_population[:pop_size]]
        # Fill remaining slots with random if needed
        while len(population) < pop_size:
            population.append(np.random.choice([-1, 1], size=N))
    else:
        population = [np.random.choice([-1, 1], size=N) for _ in range(pop_size)]

    energies = [labs_energy(s) for s in population]

    # Find best solution in initial population
    best_idx = np.argmin(energies)
    best_s = population[best_idx].copy()
    best_energy = energies[best_idx]

    if verbose:
        print(f"Initial best energy: {best_energy}")

    for gen in range(max_generations):
        # Check stopping criterion
        if target_energy is not None and best_energy <= target_energy:
            if verbose:
                print(f"Target energy reached at generation {gen}")
            break

        # Make child: combine or sample
        if np.random.random() < p_combine and pop_size >= 2:
            # Combine two parents
            idx1, idx2 = np.random.choice(pop_size, 2, replace=False)
            child = combine(population[idx1], population[idx2])
        else:
            # Sample from population
            idx = np.random.randint(pop_size)
            child = population[idx].copy()

        # Mutate child
        child = mutate(child, p_mut)

        # Run tabu search
        result, result_energy = tabu_search(child, max_iter=tabu_max_iter,
                                             tabu_tenure=tabu_tenure)

        # Update global best if improved
        if result_energy < best_energy:
            best_s = result.copy()
            best_energy = result_energy
        print(f"Generation {gen}: New best energy = {best_energy} New merit factor = {merit_factor(best_s, best_energy)}, labs_count= {labs_count}", flush=True)

        # Add result to population (replace worst member if result is better)
        worst_idx = np.argmax(energies)
        if result_energy < energies[worst_idx]:
            population[worst_idx] = result
            energies[worst_idx] = result_energy

    return best_s, best_energy, population, energies

In [129]:
# Set random seed for reproducibility
np.random.seed(42)

# Problem parameters
N = 66 # Sequence length
pop_size = 20  # Population size
max_generations = 100  # Number of generations

print(f"Running Memetic Tabu Search for LABS with N={N}")
print(f"Population size: {pop_size}, Max generations: {max_generations}")
print("-" * 50)

labs_count = 0

# Run MTS
best_s, best_energy, population, energies = memetic_tabu_search(
    N=N,
    pop_size=pop_size,
    max_generations=max_generations,
    p_mut=0.1,
    p_combine=0.5,
    tabu_max_iter=30,
    tabu_tenure=7,
    verbose=True
)

# Visualize results
# visualize_mts_results(population, energies, best_s, best_energy, N)


Generation 99: New best energy = 377 New merit factor = 5.777188328912467, labs_count= 198120
